In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MySparkApp") \
    .master("local[*]") \
    .getOrCreate()

24/08/08 20:17:21 WARN Utils: Your hostname, huytrao-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/08/08 20:17:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/08 20:17:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [19]:
# Đọc tệp CSV
orders_rdd = spark.sparkContext.textFile("Orders.csv")




In [3]:
# Lấy dòng header
header = orders_rdd.first()

# Loại bỏ dòng header khỏi RDD
orders_rdd_no_header = orders_rdd.filter(lambda line: line != header)

In [4]:
orders_rdd_no_header.count()

9426

In [5]:
orders_rdd_no_header.take(10)

['18606,Not Specified,0.01,2.88,0.5,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,Small Box,0.36,Central,Illinois,Addison,60101,5/28/2012,5/30/2012,1.32,2,5.9,88525',
 '20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,Wrap Bag,0.54,West,Washington,Anacortes,98221,7/7/2010,7/8/2010,4.56,4,13.01,88522',
 '23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,Small Box,0.37,West,Washington,Anacortes,98221,7/27/2011,7/28/2011,-47.64,7,49.92,88523',
 '23087,Not Specified,0.01,5.68,3.6,3,Bonnie Potter,Regular Air,Corporate,Office Supplies,Small Pack,0.56,West,Washington,Anacortes,98221,7/27/2011,7/28/2011,-30.51,7,41.64,88523',
 '23088,Not Specified,0,205.99,2.5,3,Bonnie Potter,Express Air,Corporate,Technology,Small Box,0.59,West,Washington,Anacortes,98221,7/27/2011,7/27/2011,998.2023,8,1446.67,88523',
 '23597,Medium,0.09,55.48,14.3,3,Bonnie Potter,Express Air,Corporate,Office Supplies,Small Box,0.37,West,Washington,

# count Orders by Region.

Using mapmethod ->  (Central,1)
(East,1)
(South,1)
(West,1)

In [6]:
map_Region = orders_rdd_no_header.map(lambda x: (x.split(",")[12],1))

In [7]:
map_Region.take(10)

[('Central', 1),
 ('West', 1),
 ('West', 1),
 ('West', 1),
 ('West', 1),
 ('West', 1),
 ('West', 1),
 ('West', 1),
 ('West', 1),
 ('West', 1)]

In [8]:
reduced_Region = map_Region.reduceByKey(lambda x,y : x+y )

In [9]:
reduced_Region.collect()

[('Central', 2899), ('West', 2284), ('East', 2289), ('South', 1954)]

In [12]:
sort_reduce_Region = reduced_Region.sortBy(lambda x : x[1],False)

In [13]:
sort_reduce_Region.collect()

[('Central', 2899), ('East', 2289), ('West', 2284), ('South', 1954)]

# Top 10 Customers with the Highest Order Volume

In [14]:
map_customer_rdd = orders_rdd_no_header.map(lambda x : (x.split(",")[6],1))

In [15]:
map_customer_rdd.take(10)

[('Janice Fletcher', 1),
 ('Bonnie Potter', 1),
 ('Bonnie Potter', 1),
 ('Bonnie Potter', 1),
 ('Bonnie Potter', 1),
 ('Bonnie Potter', 1),
 ('Bonnie Potter', 1),
 ('Ronnie Proctor', 1),
 ('Ronnie Proctor', 1),
 ('Dwight Hwang', 1)]

In [16]:
reduced_customer_rdd = map_customer_rdd.reduceByKey(lambda x,y : x + y)

In [18]:
result = reduced_customer_rdd.sortBy(lambda x : x[1],False)
result.take(5)

[('Louis Parrish', 27),
 ('Jenny Gold', 26),
 ('Sean N Boyer', 22),
 ('Leigh Burnette Hurley', 22),
 ('Andrew Gonzalez', 21)]